# LightFM

LightFMは、Pythonのためのレコメンデーションシステム構築ライブラリです。Lyst Ltdにより開発され、ハイブリッド協調フィルタリングとコンテンツベースの手法を融合していることが特徴です。
- GitHub：https://github.com/lyst/lightfm

<a href="https://colab.research.google.com/github/fuyu-quant/data-science-wiki/blob/main/recommendation/collaborative_filtering(linear)/LightFM.ipynb" target="_blank" rel="noopener noreferrer"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install lightfm

In [12]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

import numpy as np

### データの用意

In [21]:
data = fetch_movielens()
data

{'train': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 90570 stored elements in COOrdinate format>,
 'test': <943x1682 sparse matrix of type '<class 'numpy.int32'>'
 	with 9430 stored elements in COOrdinate format>,
 'item_features': <1682x1682 sparse matrix of type '<class 'numpy.float32'>'
 	with 1682 stored elements in Compressed Sparse Row format>,
 'item_feature_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1'),
 'item_labels': array(['T', 'G', 'F', ..., 'S', 'Y', 'S'], dtype='<U1')}

In [22]:
train = data["train"]
test = data["test"]

### LightFMの学習

In [23]:
model = LightFM(no_components=10,learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

### LightFMの予測
user_ids=0に対するitem_ids 1,3,4のスコア

In [27]:
predict = model.predict(user_ids=0, item_ids=[1,3,4])

user_ids 4,3,1に対するitem_ids 1,3,4のスコア

In [28]:
predict = model.predict(user_ids=np.array([4,3,1]), item_ids=[1,3,4])

### LightFMの精度

In [29]:
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10).mean()
print(f'Precision: train:{train_precision} test:{test_precision}')

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test).mean()
print(f'AUC: train:{round(train_auc, 4)} test:{test_auc}')

Precision: train:0.5913043022155762 test:0.10169672220945358
AUC: train:0.8931999802589417 test:0.856021523475647


### LightFMの学習(特徴量の追加)

In [30]:
model = LightFM(no_components=10,learning_rate=0.05, loss='bpr')

model.fit(train, item_features=data["item_features"], epochs=10)

### LightFMの精度(特徴量の追加)

In [31]:
train_precision = precision_at_k(model, train, item_features=data["item_features"], k=10).mean()
test_precision = precision_at_k(model, test, item_features=data["item_features"], k=10).mean()
print(f'Precision: train:{train_precision} test:{test_precision}')

train_auc = auc_score(model, train, item_features=data["item_features"]).mean()
test_auc = auc_score(model, test, item_features=data["item_features"]).mean()
print(f'AUC: train:{round(train_auc, 4)} test:{test_auc}')

Precision: train:0.5935313701629639 test:0.10212089121341705
AUC: train:0.8950999975204468 test:0.8594039678573608


### embeddingの取得
- ユーザーエンベディング...ユーザーのプロファイルや行動などから生成されるベクトル。これにより、似たようなユーザーが似たようなベクトル表現を持つようになります。
- アイテムエンベディング...各アイテム（例えば、映画、本、商品など）の特徴から生成されるベクトル。アイテム間の類似性を捉えることができます

In [32]:
user_embedding=model.user_embeddings
item_embedding=model.item_embeddings

In [33]:
user_embedding

array([[ 0.07701132,  0.6027149 , -0.24873352, ..., -0.06919175,
        -0.16990544,  0.03804904],
       [-0.38248426, -0.67152864,  0.8305567 , ...,  0.04864725,
        -0.5050589 , -0.29006878],
       [-0.62503785, -0.64707345,  0.6349571 , ...,  0.0126638 ,
        -0.05413153,  0.522939  ],
       ...,
       [-0.35115173, -0.01478474,  0.35465562, ..., -0.3667239 ,
        -0.203966  ,  0.01025053],
       [ 0.12668332, -0.53656095, -0.04400076, ...,  0.456537  ,
         0.48252547,  0.57658076],
       [-0.08975333,  0.8590037 , -0.701312  , ..., -0.59506   ,
        -0.1844076 , -0.2908013 ]], dtype=float32)